In [270]:
import regex as re

In [271]:
input_file = "input.py"

with open(input_file) as f:
    text = f.read()

print(text)

import json as jorre

x = 37
for k in range(x):
    print("öh" * k)
    
this_variable = "hej på dig"

a_longer123string = this_variable + "ööhö"



In [272]:
def tokenize(text) -> list[str]:
    """Tokenize a text"""
    # Possible string delimiters
    string_delims = {'"', "'", '"""', "'''"}

    # split everything
    tokens = re.findall(r"(\w+|[^\w\s]+|\s+|\n)", text)

    ## split tokens without letters or numbers or repeated operators
    tmp = []
    for t in tokens:
        if bool(re.search(r"\p{L}|\p{digit}", t)):
            tmp.append(t)
        elif len(t) > 1 and all((c == t[0] for c in t)):
            tmp.append(t)
        else:
            tmp.extend(list(t))

    tokens = tmp

    ## reunite string literals
    tmp = []
    tags = []
    current_delim = ""
    current_string = ""
    for t in tokens:
        if current_delim == "":
            if t in string_delims:
                # start a string
                current_delim = t
                current_string += t
            else:
                tmp.append(t)
                tags.append("<unk>")
        else:
            if t == current_delim:
                # break current string
                current_delim = ""
                current_string += t
                tmp.append(current_string)
                tags.append("str")
                current_string = ""
            else:
                # add to current string
                current_string += t

    tokens = tmp
    return tokens, tags


tokens, tags = tokenize(text)
print(tokens)

['import', ' ', 'json', ' ', 'as', ' ', 'jorre', '\n\n', 'x', ' ', '=', ' ', '37', '\n', 'for', ' ', 'k', ' ', 'in', ' ', 'range', '(', 'x', ')', ':', '\n', ' ', ' ', ' ', ' ', 'print', '(', '"öh"', ' ', '*', ' ', 'k', ')', '\n', ' ', ' ', ' ', ' ', '\n', 'this_variable', ' ', '=', ' ', '"hej på dig"', '\n\n', 'a_longer123string', ' ', '=', ' ', 'this_variable', ' ', '+', ' ', '"ööhö"', '\n']


## tagging

In [273]:
print("# tokens", len(tokens))

known_chars = {
    "op": r"=!%&/+-*:",
    "brac_op": r"([{",
    "brac_cl": r")]}",
}

# first: tag individual tokens

for i, token in enumerate(tokens):
    if token.isdigit():
        tags[i] = "num"
        continue
    elif bool(re.search("\s", token)):
        tags[i] = "wsp"

    for key in known_chars.keys():
        if token in known_chars[key]:
            tags[i] = key
            break


# print
for i, x in enumerate(zip(tags, tokens)):
    if x[0] != "<unk>" and x[0] != "wsp":
        print(i, x[0], repr(x[1]))

# second: tag based on context

# tokens 60
10 op '='
12 num '37'
21 brac_op '('
23 brac_cl ')'
24 op ':'
31 brac_op '('
32 str '"öh"'
34 op '*'
37 brac_cl ')'
46 op '='
52 op '='
56 op '+'
58 str '"ööhö"'


In [274]:
for x in zip(tags, tokens):
    if x[0] == "<unk>":
        print("\t".join(x))


<unk>	import
<unk>	json
<unk>	as
<unk>	jorre
<unk>	x
<unk>	for
<unk>	k
<unk>	in
<unk>	range
<unk>	x
<unk>	print
<unk>	k
<unk>	this_variable
<unk>	a_longer123string
<unk>	this_variable


In [275]:
classes = ("var", "op", "num")

## HTML cleanup

In [276]:
text = "".join(tokens)

text = re.sub(r"  ", r"&nbsp; ", text)
text = re.sub(r"\n", r"<br>\n", text)
print(text)


import json as jorre<br>
<br>
x = 37<br>
for k in range(x):<br>
&nbsp; &nbsp; print("öh" * k)<br>
&nbsp; &nbsp; <br>
this_variable = "hej på dig"<br>
<br>
a_longer123string = this_variable + "ööhö"<br>

